In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import cv2

In [5]:
def get_images(image_path):

    image = tf.image.decode_jpeg(tf.io.read_file(image_path))
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [512, 512])
    image = image[tf.newaxis, :]

    return image

In [6]:
content_image = get_images('./cry.jpg')
style_image = get_images('./demon.jpg')

In [ ]:
vgg = VGG19(
    include_top=False,
    weights='imagenet',
)
vgg.trainable = False

In [8]:
def get_model(content_layers, style_layers):
    
    content = [vgg.get_layer(layer).output for layer in content_layers]
    style = [vgg.get_layer(layer).output for layer in style_layers]

    model = Model([vgg.input], [content, style])
    return model

In [9]:
content_layers = ['block5_conv2']
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

In [10]:
model = get_model(content_layers, style_layers)

In [11]:
req_style = model(preprocess_input(style_image * 255))[1]
req_content = model(preprocess_input(content_image * 255))[0]

In [12]:
alpha = 1e4
beta = 1e-2

In [13]:
def gram_matrix(output, normalize=True):
    
    matrix = tf.linalg.einsum('nija,nijb->nab', output, output)
    shape = tf.shape(matrix)
    N = tf.cast(shape[1] * shape[2], tf.float32)
    
    if normalize:
        matrix /= N

    return matrix

In [14]:
def calc_loss(output):

    content = output[0]
    style = output[1]

    style_loss = tf.add_n([tf.reduce_mean((gram_matrix(style[i]) - gram_matrix(req_style[i]))**2) for i in range(len(style_layers))]) / len(style_layers)
    content_loss = tf.add_n([tf.reduce_mean((content[i] - req_content[i])**2) for i in range(len(content_layers))]) / len(content_layers)

    return alpha * content_loss + beta * style_loss

In [15]:
def clip(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
image = tf.Variable(content_image)

In [17]:
@tf.function
def train_step(image):
    with tf.GradientTape() as tape:
        img = preprocess_input(image*255)
        output = model(img)
        loss = calc_loss(output)

    grad = tape.gradient(loss, image)
    optimizer.apply_gradients([(grad, image)])
    image.assign(clip(image))

In [ ]:
epochs = 1000
for _ in range(epochs):
    train_step(image)

In [ ]:
image = np.array(image*255, dtype=np.uint8)
cv2.imshow(image[0])
cv2.waitKey(0)  
cv2.destroyAllWindows()